## SARVIEW get info and download to specific files

This helper notebook will introduce you on how to use the SARVIEW. Furthermore, it will also allow you to move the SARVIEW python file without using the CLI.

In [ ]:
import url_widget as url_w
notebookUrl = url_w.URLWidget()
display(notebookUrl)

In [ ]:
from IPython.display import Markdown
from IPython.display import display

notebookUrl = notebookUrl.value
user = !echo $JUPYTERHUB_USER
env = !echo $CONDA_PREFIX
if env[0] == '':
    env[0] = 'Python 3 (base)'
if env[0] != '/home/jovyan/.local/envs/rtc_analysis':
    display(Markdown(f'<text style=color:red><strong>WARNING:</strong></text>'))
    display(Markdown(f'<text style=color:red>This notebook should be run using the "rtc_analysis" conda environment.</text>'))
    display(Markdown(f'<text style=color:red>It is currently using the "{env[0].split("/")[-1]}" environment.</text>'))
    display(Markdown(f'<text style=color:red>Select the "rtc_analysis" from the "Change Kernel" submenu of the "Kernel" menu.</text>'))
    display(Markdown(f'<text style=color:red>If the "rtc_analysis" environment is not present, use <a href="{notebookUrl.split("/user")[0]}/user/{user[0]}/notebooks/conda_environments/Create_OSL_Conda_Environments.ipynb"> Create_OSL_Conda_Environments.ipynb </a> to create it.</text>'))
    display(Markdown(f'<text style=color:red>Note that you must restart your server after creating a new environment before it is usable by notebooks.</text>'))

In [ ]:
from ipyfilechooser import FileChooser
from pathlib import Path
import opensarlab_lib as asfn
from hyp3_sdk import Batch, HyP3

## 1. Login

Go to [ASF Search website](https://search.asf.alaska.edu/#/). Once you are at the website, login (or make an account).

## 2. Pick an Event

Once you log in, choose *Event* from top left-hand corner. 
![1 - choose sarview](SARVIEW_img/1_event.PNG)

You will notice many symbols on maps. Choose any that suffice your need.
![2 - map symbol](SARVIEW_img/2_map_legend.PNG)

Once you pick the symbol, you notice that there are more options at the bottom.
![3 - panel](SARVIEW_img/3_SARVIEWS_panel.PNG)

## 3. Select your items

Select the images that you wish to keep. You can do this by:
1. Clicking shopping cart mark located at the right of image you are interested (marked as 1 on image).
2. Then clicking *Downloads* located on top right-hand corner (marked as 2 on image).
![4 - select](SARVIEW_img/4_select.PNG)

## 4. Download Python File

It will prompt you a panel that lets you download the python file. Click *Data Download* and then choose *Download Python Script (.py)*.
![5 - download](SARVIEW_img/5_download.PNG)

## 5. Upload to OpenSAR

When you successfully downloaded your python file, upload them to OpenSAR lab. You can do this by pressing up-arrow button located on top left-hand corner of OpenSAR lab.

![6 - upload](SARVIEW_img/6_upload.PNG)

It will prompt you to select file that you wish to upload. Select the python file that you just downloaded.

![7 - choose script](SARVIEW_img/7_choose_script.PNG)

## 6. Run the Python File

Make sure you know where you uploaded your python file:

In [ ]:
print("Do you know where your python script is located?")

option = asfn.select_parameter(['yes', 'no'], '')
display(option)

In [ ]:
res = option.value

If you happen to forget where you uploaded the python script, run below cell and it will output the path to your python files.

**NB: Use this as a last resort as it may take a while to find location of your python file.**

In [ ]:
if res == 'no':
    rt = Path('/home/jovyan')
    py_files = list(rt.rglob('[!.]*/download-*.py'))

    print("Please choose the file you are looking for: ")
    option = asfn.select_parameter(py_files, '')
    display(option)

In [ ]:
if res == 'no':
    scriptPath = py_files[py_files.index(option.value)]
    print(scriptPath)

Right-click your python file and select *Copy Path* option to obtain path to your python script:

![8 - copy path](SARVIEW_img/8_copy_path.PNG)

In [ ]:
if res == 'yes':
    scriptPath = input('Enter the path to your python script: ')
    scriptPath = Path('/home/jovyan/')/scriptPath

At this point, you should know where you uploaded your python script to OpenSAR. If you wish to move your python file to some other directory, you can do so with cells below.

*NB: Since you're choosing destination directory, make sure your output for next cell is green.* 

In [ ]:
print("Would you like to move your python script to somewhere else?")

option = asfn.select_parameter(['yes', 'no'], '')
display(option)

In [ ]:
res = option.value

In [ ]:
if res == 'yes':
    print("Please choose a directory that you wish to move to. Make sure you don't have any file extension (i.e. output color should be green)")

    fc = FileChooser('/home/jovyan/notebooks/SAR_Training/English')
    display(fc)

In [ ]:
if res == 'yes':
    fc_path = Path(fc.selected_path)

    ! mv {scriptPath} {fc_path}
    scriptPath = fc_path/f'{scriptPath.stem}.py'

**NB: When you move item to different directory via this notebook, you may not see the change immedieately. You may refresh this page to see that the script has moved.**

In [ ]:
print('Your directory path is: ', scriptPath)

if Path(scriptPath).exists():
    print('Python file you chose exists.')
else:
    print('Python file you chose does not exist. Please verify your path to make sure you have valid script.')

## 7. Download Data

Now that you know where your python file is, you can start downloading data from your python script. 

**Select a location on where you would like to store your data. Make sure the result is colored in green.**

*NB: If you want to store in same place as this notebook, press `Select` twice.*

In [ ]:
fc = FileChooser(f'{Path.cwd()}')
display(fc)

**Enter a name of directory you wish to make. (Optional)**

In [ ]:
download_path = Path(fc.selected_path)

print("Would you like to make directory to store your downloaded data?")

option = asfn.select_parameter(['yes', 'no'], '')
display(option)

In [ ]:
res = option.value

In [ ]:
if res == 'yes':
    print('Warning: Please avoid using spaces and/or special characters other than underscores and hyphen.')
    new_dir = input('Enter a name for your new directory: ')

    download_path = Path(fc.selected_path)/new_dir

**Check if you already have directories with same name**

In [ ]:
if res == 'yes':
    if download_path.exists():
        print("Directory with same name exists. Would you like to empty this one or make new directory with different name?")
        option = asfn.select_parameter(['Make new directory with different name.', 'Empty current directory.'], '')
        display(option)

In [ ]:
if res == 'yes':
    if download_path.exists():
        if 'Empty' in option.value:
            !rm -rf {download_path}
        else:
            new_dir = input('Enter a name for your new directory: ')
            download_path = Path(fc.selected_path)/new_dir

In [ ]:
if res == 'yes':
    !mkdir {download_path}

**Authenticate and download SARVIEW data.**

In [ ]:
%run -i {scriptPath}

**Move the downloaded data to directory you just made.**

*NB:* It will download to current directory if you did not make a directory.

In [ ]:
downloaded_zips = Path.cwd().glob('S*.zip')

for d in downloaded_zips:
    !mv {d} {download_path}

**Run below to get path to directory that contains data that you just downloaded.**

In [ ]:
print(download_path)